In [ ]:
from collections import OrderedDict

def urut_dict_berdasarkan_key(dictionary):
  """Mengurutkan dictionary berdasarkan key.

  Args:
    dictionary: Dictionary yang akan diurutkan.

  Returns:
    OrderedDict: Dictionary yang sudah terurut berdasarkan key.
  """

  # Pendekatan 1: Mengubah ke list of tuples
  # sorted_items = sorted(dictionary.items())
  # sorted_dict = dict(sorted_items)

  # Pendekatan 2: Menggunakan sorted() dengan itemgetter
  # from operator import itemgetter
  # sorted_dict = dict(sorted(dictionary.items(), key=itemgetter(0)))

  # Pendekatan 3: Menggunakan OrderedDict
  sorted_dict = OrderedDict(sorted(dictionary.items()))

  return sorted_dict

# Contoh penggunaan
my_dict = {'b': 2, 'a': 1, 'c': 3}
sorted_dict = urut_dict_berdasarkan_key(my_dict)
print(sorted_dict)  # Output: OrderedDict([('a', 1), ('b', 2), ('c', 3)])

OrderedDict([('a', 1), ('b', 2), ('c', 3)])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_dir = "/content/drive/My Drive/Raja Kamen Rider/"

In [ ]:
import os

namefolder = "frame"
image_folder = base_dir + namefolder
image_files = [os.path.join(image_folder,img)
               for img in os.listdir(image_folder)
               if img.endswith(".jpg")]
print(image_files)

['/content/drive/My Drive/Raja Kamen Rider/frame/frame_641.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_642.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_643.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_644.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_645.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_646.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_647.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_648.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_649.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_650.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_651.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_652.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_653.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_654.jpg', '/content/drive/My Drive/Raja Kamen Rider/frame/frame_655.jpg', '/content/drive/My Drive/Raja Kamen Rid

In [ ]:
image_files = {int(y.replace(f"{namefolder}/","").split("_")[1].replace(".jpg", "").replace(" (1)","")):y for x,y in enumerate(image_files)}
image_files = urut_dict_berdasarkan_key(image_files)
image_files = [y for x,y in image_files.items()]

In [ ]:
!pip install onnxruntime onnxruntime-gpu insightface -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.2 MB/s eta 0:00:00


In [ ]:
from matplotlib.pyplot as plt
import cv2
import insightface

In [ ]:

from matplotlib import pyplot as plt
import cv2
import insightface
from huggingface_hub import hf_hub_download

repo_id = "ezioruan/inswapper_128.onnx"
filename = "inswapper_128.onnx"
model_path = hf_hub_download(repo_id=repo_id, filename=filename)
# providers = ["CPUExecutionProvider"]
providers = ["CUDAExecutionProvider"]

# Inisialisasi FaceAnalysis sekali saja di luar fungsi
FACE_ANALYSER = insightface.app.FaceAnalysis(
      name="buffalo_l",
      root=".", providers=providers,
      allowed_modules=["landmark_3d_68", "landmark_2d_106", "detection", "recognition"]
  )
FACE_ANALYSER.prepare(ctx_id=0, det_size=(640, 640))

  # Download model inswapper jika belum ada (lakukan sekali saja)
  # Load model inswapper
model_swap_insightface = insightface.model_zoo.get_model(model_path, providers=providers)

def swap_faces(source_img, target_img):
  """
  Melakukan swap wajah antara dua gambar menggunakan model inswapper.

  Args:
      source_img (cv2.Mat): Gambar sumber wajah yang akan ditempelkan.
      target_img (cv2.Mat): Gambar target yang akan dipasangi wajah baru.

  Returns:
      cv2.Mat: Gambar hasil swap wajah.
  """

  # Deteksi wajah pada kedua gambar
  src_faces = FACE_ANALYSER.get(source_img)
  target_faces = FACE_ANALYSER.get(target_img)

  # Pastikan kedua gambar memiliki wajah
  if not src_faces or not target_faces:
      print("Tidak ada wajah yang terdeteksi!")
      return target_img

  # Lakukan swap wajah
  img_fake = model_swap_insightface.get(
      img=target_img, target_face=target_faces[0], source_face=src_faces[0], paste_back=True
  )

  return img_fake

In [ ]:
def process_batch_images(image_files, target_img_path, folder_output="frame_fake"):
  os.makedirs(base_dir+folder_output, exist_ok=True)

  for image_file in image_files:
    target_img = cv2.imread(target_img_path)
    src_frame = cv2.imread(image_file)
    # img_fake = swap_faces(src_frame, target_img)
    img_fake = swap_faces(target_img, src_frame)
    filename = base_dir + f"{folder_output}/" + image_file.split("/")[-1]
    cv2.imwrite(filename, img_fake)
    print(filename + " Exported")

In [ ]:
process_batch_images(image_files, base_dir + "Heru~3.JPG")